<a href="https://colab.research.google.com/github/rezzycavalheiro/FERMachineLearning/blob/master/TCC1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is a prototype for my final course project. 

**Course**: Computer Science  
**University**: Pontifical Catholic University of Paraná (PUC PR)  
**Student's name**: Renata Cavalheiro da Silva  
    
**Project's title**: Classification of Facial Expressions Using Deep Learning

> **Description**: This project aims to build a Facial Expresion Recognition (FER) algorithm that presents a high accuracy and that could be adapted for psychologists and psychiatrists to use during their appointments with their pacients. The idea is that this tool would help the medical professionals to record and evaluate the expressions presented by their pacients, and use the data to check if they match what the pacient is saying. This is due to people being able to say something when they are feeling another - like someone with depressive symptoms saying they don't feel sadness, but in their expression actually showing this emotion. 






In [3]:
# ACCESSING THE FOLDER ON MY DRIVE AND GETTING ITS CONTENT

# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print("Auth Success")

# Getting the id of the folder which contains the files 
folder_id = '1vtV4IahA5vu3P8WQRRTPgRK8qPr1umKH'
lister = drive.ListFile({'q': "'%s' in parents" % folder_id}).GetList()

for item in lister:
    print(item['title']) 
    print('title: %s, mimeType: %s' % (item['title'], item['mimeType']))
    mimetypes = {
        # Drive Document files as PDF
        'application/vnd.google-apps.document': 'application/pdf',

        # Drive Sheets files as MS Excel files.
        'application/vnd.google-apps.spreadsheet': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
    }
    download_mimetype = None
    if item['mimeType'] in mimetypes:
        download_mimetype = mimetypes[file['mimeType']]
        item.GetContentFile(file['title'], mimetype=download_mimetype)

        item.GetContentFile(r'C:\Users\Renata\Desktop\googedrive\\' + item['title'], mimetype=download_mimetype)
    else: 
        item.GetContentFile(r'C:\Users\Renata\Desktop\googedrive\\' + item['title'])

Auth Success
fer2013.csv
title: fer2013.csv, mimeType: text/csv
README
title: README, mimeType: application/octet-stream
fer2013.bib
title: fer2013.bib, mimeType: text/x-bibtex


In [5]:
# READING THE FILE

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file

import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt

# Labeling the file with the characteristics

label_map = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
names=['emotion','pixels','usage']
filename = r'C:\Users\Renata\Desktop\googedrive\\fer2013.csv'
fer2013 = pd.read_csv(r'C:\Users\Renata\Desktop\googedrive\\fer2013.csv', names=names, na_filter=False)
print(fer2013.info())
im = fer2013['pixels']
fer2013.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35888 entries, 0 to 35887
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35888 non-null  object
 1   pixels   35888 non-null  object
 2   usage    35888 non-null  object
dtypes: object(3)
memory usage: 841.2+ KB
None


,emotion,pixels,usage
0,emotion,pixels,Usage
1,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
2,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
3,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
4,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
5,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
6,2,55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...,Training
7,4,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...,Training
8,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...,Training
9,3,85 84 90 121 101 102 133 153 153 169 177 189 1...,Training


In [6]:
# GETTING THE DATA AND CHECKING THE SIZE OF THE SET

def getData(filename):
    # images are 48x48
    # N = 35887
    Y = []
    X = []
    first = True
    for line in open(filename):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])

    X, Y = np.array(X) / 255.0, np.array(Y)
    return X, Y

X, Y = getData(filename)
num_class = len(set(Y))
print(num_class)

7


In [0]:
# keras with tensorflow backend
N, D = X.shape
X = X.reshape(N, 48, 48, 1)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)

In [0]:
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout ,Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.metrics import categorical_accuracy
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization

In [13]:
def my_model():
    model = Sequential()
    input_shape = (48,48,1)
    model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    # model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    # model.add(BatchNormalization())
    # model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer='adam')
    # UNCOMMENT THIS TO VIEW THE ARCHITECTURE
    #model.summary()
    
    return model
model=my_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 48, 48, 64)        1664      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 48, 48, 64)        102464    
_________________________________________________________________
batch_normalization_4 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 24, 128)       204928    
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 24, 24, 128)       409728    
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 128)      

In [0]:
path_model='model_filter.h5' # save model at this location after each epoch
K.tensorflow_backend.clear_session() # destroys the current graph and builds a new one
model=my_model() # create the model
K.set_value(model.optimizer.lr,1e-3) # set the learning rate
# fit the model
h=model.fit(x=X_train,     
            y=y_train, 
            batch_size=64, 
            epochs=10, 
            verbose=1, 
            validation_data=(X_test,y_test),
            shuffle=True,
            callbacks=[
                ModelCheckpoint(filepath=path_model),
            ]
            )

Train on 32298 samples, validate on 3589 samples
Epoch 1/10
32298/32298 [==============================] - 2996s 93ms/step - loss: 1.6957 - accuracy: 0.3384 - val_loss: 1.4725 - val_accuracy: 0.4425
Epoch 2/10
32298/32298 [==============================] - 2968s 92ms/step - loss: 1.3583 - accuracy: 0.4777 - val_loss: 1.2675 - val_accuracy: 0.5132
Epoch 3/10
32298/32298 [==============================] - 2991s 93ms/step - loss: 1.1675 - accuracy: 0.5581 - val_loss: 1.2608 - val_accuracy: 0.5216
Epoch 4/10
  832/32298 [..............................] - ETA: 45:53 - loss: 1.0149 - accuracy: 0.6298